# Variable Initialization & Imports

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
import time
import os
import pyautogui
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pytesseract as pt
import pdf2image
import PyPDF2
import re
import csv
from openpyxl import Workbook
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
import datetime
from pdf2image.exceptions import PDFPageCountError

#Setting up the Excel document and the sheets with openpyxl
wb = Workbook()
wb.active
wb['Sheet'].title = "Liner Info"
wb.create_sheet('Remarks')
wb.create_sheet('GAU Letter Info')
wb.create_sheet('Casing Info')
wb.create_sheet('Formation Info')
wb.create_sheet('TimeStamp')
#Creating names so I can write to the different sheets
LinerInfo = wb["Liner Info"]
Remarks = wb["Remarks"]
GAU = wb["GAU Letter Info"]
Casing = wb["Casing Info"]
FormationSheet = wb["Formation Info"]
Timestamp = wb["TimeStamp"]
wb.save("WellboreData.xlsx")

#Column Names
Liner_info_columns = ["API","Ro",'Liner Size', 'Hole Size','Liner Top','Liner Bottom','Cement Class',
                      'Cement Amout', 'Slurry Volume','Top of Cement','TOC Determiend']
Remarks_columns = ["API", "KOP #","Remarks"]
GAU_columns = ["API", "GAU Info"]
Casing_info_columns = ['API',"Ro",'Type of Casing','Casing Size (in.)','Hole Size','Setting Depth','Multi-Stage Tool',
                       'Multi-Stage Shoe', 'Cement Class','Cement Amount','Slurry Volume (cu.)','Top of Cement (ft.)',
                       'TOP Determined By']
Formation_info_columns = ['API','Formations','Encountere','Depth TVD','Depth MD','Is formation', "Remarks"]
Timestamp_columns = ['API', 'TimeStamp']
#Function to write the column names to the different sheets
def initialize():
    for i in range(len(Liner_info_columns)):
        LinerInfo.cell(row = 1,column = 1+i).value = Liner_info_columns[i]
    for i in range(len(Remarks_columns)):
        Remarks.cell(row = 1,column = 1+i).value = Remarks_columns[i]
    for i in range(len(GAU_columns)):
        GAU.cell(row = 1,column = 1+i).value = GAU_columns[i]
    for i in range(len(Casing_info_columns)):
        Casing.cell(row = 1,column = 1+i).value = Casing_info_columns[i] 
    for i in range(len(Formation_info_columns)):
        FormationSheet.cell(row = 1,column = 1+i).value = Formation_info_columns[i]
    for i in range(len(Timestamp_columns)):
        Timestamp.cell(row = 1, column = 1+i).value = Timestamp_columns[i]
wb.save("WellboreData.xlsx")

#row list that tracks the new row available for each sheet
#row = [2,2,2,2,2,2]

#Strings necessary for navigating through website           
Chrome_PATH = '/Users/stefan/Documents/2021 Internship/California Oil Info/Chromedrive/chromedriver'
JS_clearcounty = 'document.querySelector("body > table:nth-child(2) > tbody > tr > td > form > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(10) > td:nth-child(2) > input[type=text]:nth-child(1)").value = '
JS_clearID = "document.querySelector('#apiNoSuffixArg').value = "
JD_downloadname = "document.querySelector('downloads-manager').shadowRoot.querySelector('#downloadsList downloads-item').shadowRoot.querySelector('div#content  #file-link').text"
JS_downloadpercent = "document.querySelector('downloads-manager').shadowRoot.querySelector('#downloadsList downloads-item').shadowRoot.querySelector('#progress').value"
pytesseract_path = '/Users/stefan/opt/anaconda3/bin/pytesseract'

#Grab API Numbers from other Excel document
#df = pd.read_excel("Excel Documents/Texas Casing API.xlsx")
#API = df['API'].tolist()
#API = [sub.replace('-','') for sub in API]


df = pd.read_excel("Excel Documents/800 APIs for GAU LETTERS.xlsx")
API = df['API'].tolist()
API = [str(int(x/10000)) for x in API]


#Change some of the selenium chrome settings for our use
URL = 'http://webapps2.rrc.texas.gov/EWA/wellboreQueryAction.do'
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
"download.default_directory": "/Users/stefan/Documents/2021 Internship/API Matching/RRC_Data",
"download.prompt_for_download": False, 
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True 
})


# Main Function

In [6]:
def main(beginning,ending):
    filenames = os.listdir('RRC_Data')
    for k in filenames:
        if k == '.DS_Store':
            pass
        os.remove('RRC_Data/'+k)
    driver = webdriver.Chrome(Chrome_PATH, options=options)
    driver.get(URL)
    for i in range(beginning,ending):
        # ------------------------------- SEARCH Section -----------------------------
        driver.execute_script(JS_clearcounty+'"'+str(API[i][2:5]).zfill(3)+'"')
        driver.execute_script(JS_clearID+'"'+str(API[i][5:]).zfill(5)+'"')
        driver.find_element_by_xpath('//*[@id="scheduleTypeArg3"]').click()
        element = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr/td/form/table/tbody/tr[2]/td/table/tbody/tr[3]/td/input[1]')
        time.sleep(1)
        ActionChains(driver).key_down(Keys.COMMAND).click(element).key_up(Keys.COMMAND).perform()
        driver.switch_to.window(driver.window_handles[1])
        try:
            #If this passes without raising an error, it means the API entered is incorrect
            driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[3]/td/ul/li[1]/font')
            #Press the reset button if still here
            time.sleep(5)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except (NoSuchElementException,StaleElementReferenceException,ElementNotInteractableException):
            # ------------------------------- API Section -----------------------------
            #Click on first API# Completion
            driver.find_element_by_xpath('//*[@id="apiRelatedLinks:'+str(API[i][2:]).zfill(8)+'"]').click()
            driver.find_element_by_xpath('//*[@id="apiRelatedLinks:'+str(API[i][2:]).zfill(8)+'"]/option[4]').click()
            print("API Number:", API[i], i)

            # ------------------------------- Tracking Number Section -----------------------------
            #Now we need to somehow determine how many links there are
            WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "/html/body/table[5]/tbody/tr/td/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]")))
            for j in range(1,6):
                try:
                    driver.find_element_by_xpath('/html/body/table[5]/tbody/tr/td/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr['+str(j)+']')
                    iterations = j - 2
                except (NoSuchElementException):
                    pass
            #Click on tracking numbers
            for z in range(3,iterations+3):
                #Click next link in new tab and switch to that tab
                element = driver.find_element_by_xpath('/html/body/table[5]/tbody/tr/td/form/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr['+str(z)+']/td[1]/a')
                ActionChains(driver).key_down(Keys.COMMAND).click(element).key_up(Keys.COMMAND).perform()
                driver.switch_to.window(driver.window_handles[2])

                # ------------------------------- Form Summary Section -----------------------------
                #Grab info about where the W-2, G-1, and GAU Letter's are
                indices = []
                dictionary = {}
                table = driver.find_element_by_xpath('/html/body/table[5]/tbody/tr/td/form/table/tbody/tr/td/table[2]/tbody/tr[2]/td[1]/table/tbody/tr[2]/td/table/tbody').text
                table = table.split('\n')
                table = list(filter("View Report".__ne__,table))
                table = list(filter("Certified".__ne__,table))
                table = list(filter("Form Status View Report".__ne__,table))
                indices = [i for i, x in enumerate(table) if x == 'W-2'] + [i for i, x in enumerate(table) if x == 'GAU LETTER']+ [i for i, x in enumerate(table) if x == 'G-1']

                #Create dictionary to pair table row and doc type
                for d in range(len(indices)):
                    dictionary[str(indices[d])] = table[indices[d]]

                for p in range(len(indices)):
                    # ------------------------------- Individual Document Section -----------------------------
                    #Click on download link
                    time.sleep(0.5)
                    driver.find_element_by_xpath('/html/body/table[5]/tbody/tr/td/form/table/tbody/tr/td/table[2]/tbody/tr[2]/td[1]/table/tbody/tr[2]/td/table/tbody/tr['+str(indices[p]+2)+']/td[3]/a').click()
                    
                    #Wait until file has been downloaded
                    time.sleep(3)
                    while (len(os.listdir('RRC_Data'))==0):
                        time.sleep(1)

                    #Read the file name of downloaded file
                    #Consider chaning the filename of interest to filenames[-1] in case of failed deleting
                    filenames = os.listdir('RRC_Data') 

                    #Identify document type and read it
                    document = dictionary[str(indices[p])]
                    driver.switch_to.window(driver.window_handles[2])
                    if (document == "W-2"):
                        time.sleep(1)
                        #G1W2Reader(filenames[0], API[i])
                    elif (document == "GAU LETTER"):
                        time.sleep(1)
                        GAU_reader(filenames[0], API[i])
                    elif (document == "G-1"):
                        time.sleep(1)
                        #G1W2Reader(filenames[0], API[i])
                    #Delete document
                    os.remove('RRC_Data/'+filenames[0])
                    #Record Timestamp
                    Timestamp.cell(row = row[5], column = 1).value = API[i]
                    Timestamp.cell(row = row[5], column = 2).value = '{:%H:%M:%S}'.format(datetime.datetime.now())
                    row[5] = row[5] + 1
                    #Save changes to doc
                    wb.save("WellboreData.xlsx")
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
            time.sleep(1)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.5)
        wb.save("WellboreData.xlsx")
    driver.quit()
    print("Finished!")

# GAU Letter Read & Write Function

In [7]:
def GAU_reader(pdf_name, current_API):
    content = []
    data = ''
    info = ''
    s,f = [],[]
    try:
        pages = pdf2image.convert_from_path(pdf_path= 'RRC_Data/'+pdf_name)
        for i in range(len(pages)):
            pages[i].save(pdf_name+ str(i)+'.jpg')
            content.append(pt.image_to_string(pages[i],lang='eng'))
            data = data + content[i]
        os.remove(pdf_name+ str(i)+'.jpg')
        start = "recommends:"
        finish = "Groundwater Advisory"
        try:
            s = [i.start() for i in re.finditer(start,data)][0]
            f = [i.start() for i in re.finditer(finish,data)][-1]
            info = data[s+len(start):f].replace('\n','')

        except IndexError:
            try: 
                f = [i.start() for i in re.finditer("Sincerely",data)][-1]
                info = data[s+len(start):f].replace('\n','')
            except IndexError:
                print("GAU Index Error:", s,f)
            pass
        print('---------------------------------- GAU start -----------------------------')
        print(info)
        print('----------------------------------- GAU end ------------------------------')
        #write data to GAU sheet
        GAU.cell(row = row[2], column= 1).value = current_API
        GAU.cell(row = row[2], column= 2).value = info
        row[2] = row[2]+1
    except PDFPageCountError:
        pass 

# G1W2 Document Read Function

In [8]:
def G1W2Reader(pdf_name, current_API):
    content = []
    data = ''
    info = ''
    try:
        pages = pdf2image.convert_from_path(pdf_path= 'RRC_Data/'+pdf_name)
        for i in range(len(pages)):
            pages[i].save(pdf_name+ str(i)+'.jpg')
            content.append(pt.image_to_string(pages[i],lang='eng'))
            data = data + content[i]
            os.remove(pdf_name+ str(i)+'.jpg')
        print('---------------------------------- G1W2 start ----------------------------')
        s = [i.start() for i in re.finditer("LINER RECORD",data)]
        f = [i.start() for i in re.finditer("TUBING RECORD",data)]
        Organize_Liner(data[s[0]+len("LINER RECORD"):f[0]],current_API)
        Remarks_func(pdf_name, current_API)
        s = [i.start() for i in re.finditer("CASING RECORD",data)]
        f = [i.start() for i in re.finditer("LINER RECORD",data)]
        Casing_Info(data[s[0]+len("CASING RECORD"):f[0]], current_API)
        s = [i.start() for i in re.finditer("FORMATION RECORD",data)]
        f = [i.start() for i in re.finditer("REMARKS",data)]
        Formation(data[s[0]+len("FORMATION RECORD"):f[0]], current_API)
        print('---------------------------------- G1W2 end ------------------------------')
    except PDFPageCountError:
        pass 

# Remarks Data Function

In [9]:
def Remarks_func(PDF_Name, current_API):
    info = ''
    pdfFileObj = open('RRC_Data/'+PDF_Name, 'rb') 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    numpages = pdfReader.numPages
    for i in range(numpages):
        pageObj = pdfReader.getPage(i)
        info = info +'\n'+'\n'+ pageObj.extractText()
    pdfFileObj.close()
    print(info)
    j = [i.start() for i in re.finditer("REMARKS",info)]
    k = [i.start() for i in re.finditer("RRC REMARKS",info)]
    string = info[j[0]+len("REMARKS"): k[0]]    
    address = string.find('KOP')+len('KOP')
    x = string[address:address+15]
    numeric_filter = filter(str.isdigit, x)
    numeric_string = "".join(numeric_filter)
    if numeric_string == '':
        numeric_string = 'NA'
    
    if '4Page' in string:
        string = string[:string.index('4Page')]
    print("String:", string)
    Remarks.cell(row = row[1],column = 1).value = current_API
    Remarks.cell(row = row[1],column = 2).value = numeric_string
    Remarks.cell(row = row[1],column = 3).value = string
    row[1] = row[1] + 1 
    print("Remarks:",string)

# Liner Data Function

In [10]:
def Organize_Liner(string, current_API):
    print("----------- Liner Data ------------")
    string = string.split('\n')
    numeric = 0
    #Remove all unnecessary entries
    for i in range(100):
        try:
            x = string[i].replace(" ","")
            if  x == '':
                string[i] = ''
        except IndexError:
            pass
    while '' in string:
        string.remove('')
    string = ''.join(string[3:])
    for k in string:
        if k.isnumeric():
            numeric = numeric + 1
    if (numeric> 15):
        #string = string[string.index("Determined")+len("Determined"):]
        liner_data = string.split(' ')
        while("" in liner_data) :
            liner_data.remove("")
        for char in liner_data[-1]:
            if char.isnumeric():
                liner_data[-2] = liner_data[-2] + char
                liner_data[-1] = liner_data[-1].replace(char,'')
        #Read where the slashes are
        slashes = []
        for j in range(len(liner_data)):
            if '/' in liner_data[j]:
                slashes.append(j)
        #Based on where the slashes are, merge certain list entries
        if slashes == [2,4]:
            liner_data[1:3] = [''.join(liner_data[1:3])]
            liner_data[2:4] = [''.join(liner_data[2:4])]
        elif slashes == [1,3]:
            liner_data[2:4] = [''.join(liner_data[2:4])]
        elif slashes == [2,3]:
            liner_data[1:3] = [''.join(liner_data[1:3])]
        elif slashes == [2]:
            if len(liner_data) > 3:
                pass
            else:
                liner_data[1:3] = [''.join(liner_data[1:3])]
        elif slashes == [3]:
            liner_data[2:4] = [''.join(liner_data[2:4])]
        
        #re-read slashes
        slashes = []
        for j in range(len(liner_data)):
            if '/' in liner_data[j]:
                slashes.append(j)
        #Add a space in the liner & hole size
        if slashes == [1,2]:
            liner_data[1] = liner_data[1][:liner_data[1].find('/')-1]+ ' '+ liner_data[1][liner_data[1].find('/')-1:]
            liner_data[2] = liner_data[2][:liner_data[2].find('/')-1]+ ' '+ liner_data[2][liner_data[2].find('/')-1:] 
    else:
        liner_data = ["NA","NA","NA","NA", "NA", "NA","NA","NA", "NA","NA"]
    print(liner_data)    
    LinerInfo.cell(row = row[0], column = 1).value = current_API
    for i in range(2,12):
        try:
            LinerInfo.cell(row = row[0], column = i).value = liner_data[i-2]
        except IndexError:
            pass
    row[0] = row[0]+1


# Casing Data Function

In [11]:
def Casing_Info(string, current_API):
    print("----------- Casing Data ------------")
    string = string.split('\n')
    info = []
    #Remove all unnecessary entries
    for i in range(100):
        try:
            x = string[i].replace(" ","")
            if  x == '':
                string[i] = ''
        except IndexError:
            pass
    while '' in string:
        string.remove('')
    #Delete first 2 lines from list (casing column titles)
    string = string[3:]
    for i in range(len(string)):
        string[i] = string[i].split(' ')
    #Combine words in list from the back
    #Sometimes the Top of Cement has words and it messes it up
    for k in range(len(string)):
        for i in reversed(string[k]):
            if i.isnumeric():
                string[k][string[k].index(i)+1:] = [' '.join(string[k][string[k].index(i)+1:])]
                break
            elif i.isupper():
                string[k][string[k].index(i)+1:] = [' '.join(string[k][string[k].index(i)+1:])]
                break
            elif isinstance(i,float):
                string[k][string[k].index(i)+1:] = [' '.join(string[k][string[k].index(i)+1:])]
                break
    #Combine words in list from the front
    #Somtimes the casing size doesn't have a slash and this messes up the whole thing
    for k in range(len(string)):
        for i in (string[k][1:]):
            if i.isnumeric():
                string[k][1:string[k].index(i)] =[' '.join(string[k][1:string[k].index(i)])]
                break
            elif '/' in i:
                string[k][1:string[k].index(i)] =[' '.join(string[k][1:string[k].index(i)])]
                break
    #Clear out any list entries that have 3 or fewer items
    for k in string:
        if len(k) <= 3:
            del string[string.index(k)]
    for k in string:
        if len(k) <= 3:
            del string[string.index(k)]
    for k in string:
        if len(k) <= 3:
            del string[string.index(k)]
    #Replace any random characters that might be taking up an entry
    for k in range(len(string)):
        for i in string[k]:
            string[k][string[k].index(i)] = i.replace("=",'').replace('+','').replace("@",'').replace("$",'').replace("%",'').replace("(",'').replace(")",'').replace("|",'').replace("/",'').replace('&','').replace("^",'').replace("-",'')
        while ('' in string[k]):
            string[k].remove('')
    #Add spaces 1 space before '/'
    for i in string:
        try:
            i[2] = i[2][:i[2].find('/')-1]+ ' '+ i[2][i[2].find('/')-1:]
            i[3] = i[3][:i[3].find('/')-1]+ ' '+ i[3][i[3].find('/')-1:]
        except IndexError:
            pass
        
    #Assumption: if there is a missing record, it is the Multi-Stage shoe record
    #Assumption: if there are two missing records, it is the Multi-Stage shoe and Multi-Stage Tool records
    for k in range(len(string)):
        if len(string[k]) == 12:
            pass
        elif len(string[k]) == 11:
            string[k].insert(6, "NA")
        elif len(string[k]) == 10:
            string[k].insert(5, "NA")
            string[k].insert(6, "NA")
        elif len(string[k]) < 10:
            for a in range(10-len(string[k])):
                string[k].insert(5+a, "NA")
    
    #Sort elements my Ro number
    def takeFirst(elem):
        return(elem[0])
    string.sort(key=takeFirst)
    
    #Write to Excel file
    Casing.cell(row = row[3], column = 1).value = current_API
    for k in string:
        o = 0
        for i in k:
            Casing.cell(row = row[3], column = o+2).value = i
            o=o+1
        row[3] = row[3]+1 
    print(string)

# Formation Data Function

In [12]:
def Formation(string, current_API):
    print("----------- Formation Data ------------")
    string = string.split('\n')
    #Remove all unnecessary entries
    for i in range(100):
        try:
            x = string[i].replace(" ","")
            if  x == '':
                string[i] = ''
        except IndexError:
            pass
    while '' in string:
        string.remove('')
    string = string[2:]
    try:
        end = [i for i in string if 'producing interval of' in i] + [i for i in string if 'Page 3' in i] + [i for i in string if 'Page 4' in i]
        end.sort()
        string = string[:string.index(end[0])]
    except IndexError:
        pass
    for i in range(len(string)):
        string[i] = string[i].split(' ')
    for k in range(len(string)):
        for i in (string[k][:]):
            if (i == "Yes") | (i == "No"):
                string[k][:string[k].index(i)] =[' '.join(string[k][:string[k].index(i)])]
                break
    for k in range(len(string)):
        for i in reversed(string[k]):
            if (i == "No"):
                string[k][string[k].index(i)+2:] = [' '.join(string[k][string[k].index(i)+2:])]
                break
    #Clear out any list entries that have 3 or fewer items
    for k in string:
        if len(k) <= 2:
            del string[string.index(k)]
    for k in string:
        if len(k) <= 2:
            del string[string.index(k)]
    for k in range(len(string)):
        if len(string[k]) >= 7:
            string[k][6:8] =  [' '.join(string[k][6:8])]
        elif len(string[k]) == 5:
            string[k].insert(5, "NA")
        elif len(string[k]) == 4:
            string[k].insert(2, "NA")
            string[k].insert(3, "NA")
    for k in range(len(string)):
        while '' in string[k]:
            string[k].remove('')
    FormationSheet.cell(row = row[4], column = 1).value = current_API
    for k in string:
        o = 0
        for i in k:
            i = ILLEGAL_CHARACTERS_RE.sub(r'',i)
            FormationSheet.cell(row = row[4], column = o+2).value = i
            o =o+1
        row[4] = row[4]+1 
    print(string)
    

# Document Reset Function

In [13]:
def ClearData():
    LinerInfo.delete_cols(1,100)
    LinerInfo.delete_rows(1,100)
    Remarks.delete_cols(1,100)
    Remarks.delete_rows(1,100)
    GAU.delete_cols(1,100)
    GAU.delete_rows(1,100)
    Casing.delete_cols(1,100)
    Casing.delete_rows(1,100)
    FormationSheet.delete_cols(1,100)
    FormationSheet.delete_rows(1,100)
    wb.save("WellboreData.xlsx")
    

In [37]:

print(row)
main(750,1700)

[2, 2, 1097, 2, 2, 2305]
API Number: 4247538154 750
---------------------------------- GAU start -----------------------------
 The interval from the land surface to the base of the Santa Rosa, which is estimated to occur at a depth of 1300 feet,must be protected.In addition, the Capitan Reef must be protected if it is penetrated.This recommendation is applicable to all wells within a radius of 200 feet of this location.Please send Gamma/Porosity log of this well when it is available.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.Unless stated otherwise, this recommendation is for normal drilling, production, and plugging operations only.This determination is based on information provided when the application was submitted on 06/16/2020. If the locationinformation has changed, you must contact the Groundwater Advisory Unit, and submit a new application if necessary.If you have questions, please contact us at 51

API Number: 4247538151 753
---------------------------------- GAU start -----------------------------
 The interval from the land surface to the base of the Rustler, which is estimated to occur at a depth of 1300 feet, mustbe protected.In addition, the Capitan Reef must be protected if it is penetrated.This recommendation is applicable to all wells within a radius of 200 feet of this location.Please send Gamma/Porosity log of this well when it is available.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.Unless stated otherwise, this recommendation is for normal drilling, production, and plugging operations only.This determination is based on information provided when the application was submitted on 06/16/2020. If the locationinformation has changed, you must contact the Groundwater Advisory Unit, and submit a new application if necessary.If you have questions, please contact us at 512-463-2741 or gau@rrc.texas.

API Number: 4247538011 759
---------------------------------- GAU start -----------------------------
 The interval from the land surface to a depth of 450 feet, and the Rustler, the top of which is estimated to occur from 800to 900 feet depth and the base of which is estimated to occur from 1300 to 1350 feet depth by reconnaissance-levelevaluation, must be protected.This recommendation is applicable for all wells drilled in this NE/2 of sec. 11.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.This recommendation is for normal drilling, production, and plugging operations only. It does not apply to saltwater disposaloperation into a nonproductive zone (RRC Form W-14).This determination is based on information provided when the application was submitted on 02/20/2017. If the locationinformation has changed, you must contact the Groundwater Advisory Unit, and submit a new application if necessary.If you have questi

API Number: 4247537952 764
---------------------------------- GAU start -----------------------------
 The interval from the land surface to the base of the Santa Rosa, which is estimated to occur at a depth of 1300 feet,must be protected.In addition, the Capitan must be protected if it is penetrated. The top of the Capitan is estimated to occur at 3500 feetdepth and the base is estimated to occur at 5025 feet depth by reconnaissance-level evaluation.This recommendation is applicable to all wells within a radius of 200 feet of this location.Please send Gamma/Porosity log of this well when it is available.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.Unless stated otherwise, this recommendation is for normal drilling, production, and plugging operations only.This determination is based on information provided when the application was submitted on 03/05/2019. If the locationinformation has changed, you must cont

---------------------------------- GAU start -----------------------------
 The interval from the land surface to a depth of 250 feet, and the Rustler from 750 to 1250 feet must be protected.This recommendation is applicable to all wells within a radius of 2000 feet of this location.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.Unless stated otherwise, this recommendation is for normal drilling, production, and plugging operations only.This determination is based on information provided when the application was submitted on 10/08/2018. If the locationinformation has changed, you must contact the Groundwater Advisory Unit, and submit a new application if necessary.If you have questions, please contact us at 512-463-2741 or gau@rrc.texas.gov.
----------------------------------- GAU end ------------------------------
---------------------------------- GAU start -----------------------------
 The interval from the

API Number: 4247537681 777
---------------------------------- GAU start -----------------------------
 The interval from the land surface to a depth of 1050 feet must be protected.Please send Gamma Ray/Porosity log of this well when it is available.This recommendation is applicable to all wells within a radius of 500 feet of this location.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.This recommendation is for normal drilling, production, and plugging operations only. It does not apply to saltwater disposaloperation into a nonproductive zone (RRC Form W-14).This determination is based on information provided when the application was submitted on 06/09/2016. If the locationinformation has changed, you must contact the Groundwater Advisory Unit, and submit a new application if necessary.If you have questions, please contact us at 512-463-2741 or gau@rrc.texas.gov.
----------------------------------- GAU end ----

API Number: 4247537662 784
---------------------------------- GAU start -----------------------------
 Water-bearing strata from the land surface to a depth of 200 feet and the Rustler, which is estimated to occur from 700to 1150 feet must be protected.This recommendation is applicable to all wells within a radius of 1000 feet of this location.Please send Gamma/Porosity log of this well when it is available.Note: Unless stated otherwise, this recommendation is intended to apply only to the subject well and not for area-wide use.Unless stated otherwise, this recommendation is for normal drilling, production, and plugging operations only.This determination is based on information provided when the application was submitted on 08/29/2018. If the locationinformation has changed, you must contact the Groundwater Advisory Unit, and submit a new application if necessary.If you have questions, please contact us at 512-463-2741 or gau@rrc.texas.gov.
----------------------------------- GAU end -

IndexError: list index out of range

In [38]:
wb.save("WellboreData.xlsx")